In [2]:
#%% Import moduldes

import numpy as np
import shap
import pandas as pd
from sklearn.linear_model import LogisticRegression
import sys

from helper2 import *
from helper2_dep import *
from helper2_indep import *
from helper2_shapley_sampling import *
from helper4_kshap import *
from helper_sim2 import *
from os.path import join
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split

n_pts = 40
nsim_per_point = 50
M=1000
n_samples_per_perm=[10,1]  # 1st for Kernel Shap, 2nd for Shapley Sampling
K=50
n_boot=100


/Users/jeremygoldwasser/opt/anaconda3/envs/shap/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
name = "bank"
mod = "glm"
fname = name+"_"+mod
X_train, y_train, X_test, y_test, mapping_dict = loaddata(name)

mean = X_train.mean(axis=0)
std = X_train.std(axis=0)
X_train = (X_train - mean) / std
# X_val = (X_val - mean) / std
X_test = (X_test - mean) / std
d = X_train.shape[1]

feature_means = np.mean(X_train, axis=0)
cov_mat = np.cov(X_train, rowvar=False)
    
cov_mat = correct_cov(cov_mat,Kr=10000)

M_linear = 1000
D_matrices = make_all_lundberg_matrices(M_linear, cov_mat)


#%% Train model

model, gradfn, hessfn, sds = fitmodgradhess(mod,X_train,y_train,X_test,y_test)


#%% Run simulation

X_locs = X_test[np.random.choice(X_test.shape[0],n_pts,replace=False)]

Class imbalance: 88.29904888299049
Estimation accuracy: 90.22340190223403


In [52]:
%run helper4_kshap
n_pts = X_locs.shape[0]
X = X_train
d = X.shape[1]

feature_means = np.mean(X, axis=0)

kshaps_indep = []
kshaps_dep = []
sss_indep = []
sss_dep = []
i = 20
j = 0
xloci = X_locs[i].reshape((1,d))

gradient = gradfn(model,xloci,sds)
hessian = hessfn(model,xloci,sds)

shap_CV_true_indep = compute_true_shap_cv_indep(xloci, gradient, hessian, feature_means, cov_mat, mapping_dict)
shap_CV_true_dep = linear_shap_vals(xloci, D_matrices, feature_means, gradient,mapping_dict=mapping_dict)

sims_kshap_indep = []
sims_kshap_dep = []
np.random.seed(2)
for j in range(10):
    # independent_features=True
    # obj_kshap_indep = cv_kshap_compare(model, X, xloci,
    #                     independent_features,
    #                     gradient, hessian,
    #                     shap_CV_true=shap_CV_true_indep,
    #                     M=M, n_samples_per_perm=n_samples_per_perm[0], K = K, n_boot=n_boot,
    #                     mapping_dict=mapping_dict)  
    # sims_kshap_indep.append(obj_kshap_indep)
    independent_features=False
    obj_kshap_dep = cv_kshap_compare(model, X, xloci,
                    independent_features,
                    gradient, 
                    shap_CV_true=shap_CV_true_dep,
                    M=M, n_samples_per_perm=n_samples_per_perm[0], K = K, n_boot=n_boot,
                    mapping_dict=mapping_dict)  
    sims_kshap_dep.append(obj_kshap_dep)

covariance is not positive-semidefinite.
covariance is not positive-semidefinite.
covariance is not positive-semidefinite.
covariance is not positive-semidefinite.
covariance is not positive-semidefinite.
covariance is not positive-semidefinite.
divide by zero encountered in divide
invalid value encountered in matmul
divide by zero encountered in divide
invalid value encountered in matmul
covariance is not positive-semidefinite.
covariance is not positive-semidefinite.
covariance is not positive-semidefinite.
covariance is not positive-semidefinite.


In [54]:
# obj_kshap_indep
# return vanilla_kshap_model, vanilla_kshap_CV, final_kshap_est_boot, corrs_boot, final_kshap_est_grouped, corrs_grouped, final_kshap_est_wls, corrs_wls
kshaps_indep = np.array(sims_kshap_indep)
np.mean(kshaps_indep[:,7,:], axis=0) # Mean corrs for all features
print(kshaps_indep.shape)
vars_kshap_indep = np.var(kshaps_indep,axis=0)
print(1-vars_kshap_indep[2]/vars_kshap_indep[0]) # Empirical variance reductions - bootstrapping
print(1-vars_kshap_indep[6]/vars_kshap_indep[0]) # LS
print(np.sum(np.isnan(kshaps_indep[:,6:,:]))) # No NAs in LS columns


IndexError: too many indices for array: array is 1-dimensional, but 3 were indexed

(0,)

In [57]:
kshaps_dep = np.array(sims_kshap_dep)
np.mean(kshaps_dep[:,7,:], axis=0) # Mean corrs for all features
print(kshaps_dep.shape)
vars_kshap_dep = np.var(kshaps_dep,axis=0)
print(1-vars_kshap_dep[2]/vars_kshap_dep[0]) # Empirical variance reductions - bootstrapping
print(1-vars_kshap_dep[6]/vars_kshap_dep[0]) # LS


(10, 8, 16)
[ 0.35283415  0.62007621  0.72163247  0.48711706  0.93208355  0.76217623
  0.76849598  0.49928625  0.80556332  0.74672616 -0.05344396  0.64787705
  0.69603208  0.71939765 -0.88379903  0.53647138]
[ 0.4298634   0.60062281  0.73987711  0.51046418  0.92528976  0.83268213
  0.77403821  0.50169739  0.80003621  0.80358091  0.00710952  0.54792657
  0.22615709  0.74408673 -2.66772811  0.55641018]


In [ ]:
np.sum()